Construction of curated bird banding dataset from https://www.sciencebase.gov/catalog/item/60914db3d34e791692e13a22

Species being targeted in this dataset construction are selected species of plover: Piping Plover, Black-bellied Plover, Wilson's Plover, Semipalmated Plover, and Snowy Plover. The datasets available for download are split into 10 groups based on taxonomic order. All plovers being targeted in this dataset construction were contained within dataset 4. Data was filtered using powerquery in Excel to generate a subset of the data only containing desired species. 

Sciencebase provides a series of reference datasets/lookup tables to accompany the datasets, which contain valuable metadata associated with codes used in the primary dataset. These lookup tables will be leveraged to enrich the extracted subset of data and generate a more complete table, which will include well-labeled metadata.

Cleaning procedures will be performed on the dataset including: standardizing casing across strings + removing whitespace and newlines in select columns, correcting typos, standardizing dates, removing invalid data, and addressing missing data. 

In [1]:
import pandas as pd
import glob

In [2]:
# get raw data -- note common forms of N/A are being addressed while reading into the dataframe
missing_values = ["n/a", "na", "N/A", "NA" ,"--", "-"]
raw_data = pd.read_csv("Input_Files/raw_data_plover_banding.csv", na_values = missing_values)
pd.set_option('display.max_columns', None) # lots of columns, need to set to display max
raw_data.head()

,BAND,ORIGINAL_BAND,OTHER_BANDS,EVENT_TYPE,EVENT_DATE,EVENT_DAY,EVENT_MONTH,EVENT_YEAR,ISO_COUNTRY,ISO_SUBDIVISION,LAT_DD,LON_DD,COORDINATES_PRECISION_CODE,BAND_TYPE_CODE,SPECIES_NAME,SPECIES_ID,BIRD_STATUS,EXTRA_INFO_CODE,AGE_CODE,SEX_CODE,PERMIT,BAND_STATUS_CODE,HOW_OBTAINED_CODE,WHO_OBTAINED_CODE,REPORTING_METHOD_CODE,PRESENT_CONDITION_CODE,MIN_AGE_AT_ENC,RECORD_SOURCE,Column1
0,B07104722502,B07104722502,NaN,B,6/13/2019,13,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3.0,1.0,4.0,0.0,P6726807,0.0,NaN,NaN,NaN,NaN,NaN,B,NaN
1,B07104722582,B07104722582,NaN,B,6/21/2019,21,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3.0,1.0,4.0,0.0,P6726807,0.0,NaN,NaN,NaN,NaN,NaN,B,NaN
2,B07234100054,B07234100054,NaN,B,8/2/2016,2,8,2016,US,US-ND,46.75000,-100.75000,10,41,Piping Plover,2770,3.0,69.0,4.0,0.0,P2962009,0.0,NaN,NaN,NaN,NaN,NaN,B,NaN
3,B07234100061,B07234100061,NaN,B,6/2/2017,2,6,2017,US,US-ND,47.75000,-102.41667,10,41,Piping Plover,2770,3.0,25.0,1.0,0.0,P2962009,0.0,NaN,NaN,NaN,NaN,NaN,B,NaN
4,B07234100070,B07234100070,NaN,B,5/30/2017,30,5,2017,US,US-ND,47.91667,-102.41667,10,41,Piping Plover,2770,3.0,25.0,1.0,0.0,P2962009,0.0,NaN,NaN,NaN,NaN,NaN,B,NaN


In [3]:
# several columns use codes in reference to values in lookup tables
# some interpreted as floats, so changing all to strings and removing tailing decimals
convert_age_dict = {'BIRD_STATUS': str,
                    'EXTRA_INFO_CODE': str,
                    'AGE_CODE': str,
                    'SEX_CODE': str,
                    'BAND_STATUS_CODE': str,
                    'COORDINATES_PRECISION_CODE': str,
                    'SPECIES_ID': str,
                    'HOW_OBTAINED_CODE': str,
                    'WHO_OBTAINED_CODE': str,
                    'REPORTING_METHOD_CODE': str,
                    'PRESENT_CONDITION_CODE': str,
                    'MIN_AGE_AT_ENC': str       
               }
working_df = raw_data.astype(convert_age_dict, errors = 'ignore').replace('\.0', '', regex=True)

In [4]:
# trimming whitespace from beginning and end of each string column
# note that any case standardization will be performed AFTER enrichment of the primary dataset using the reference tables
working_df = working_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [5]:
# get reference tables to be used to enrich primary dataset - will iterate through each table to understand context and structure
# as well as which fields can be added to the primary dataset 
filenames = glob.glob("Input_Files/lookup_tables/*.csv")
d = {}
for x in filenames:
    print(x)
    adj_key = x.replace("Input_Files/lookup_tables\\", '')
    adj_key = adj_key.replace('.csv', '')
    d[adj_key] = pd.read_csv(x)

# print list of reference tables -- determine which will be used to translate codes and enrich the primary dataset
for key, value in d.items():
    print(key)

Input_Files/lookup_tables\age.csv
Input_Files/lookup_tables\band_status.csv
Input_Files/lookup_tables\band_type.csv
Input_Files/lookup_tables\bird_status.csv
Input_Files/lookup_tables\coordinates_precision.csv
Input_Files/lookup_tables\country_state.csv
Input_Files/lookup_tables\event_type.csv
Input_Files/lookup_tables\extra_info.csv
Input_Files/lookup_tables\how_obtained.csv
Input_Files/lookup_tables\inexact_dates.csv
Input_Files/lookup_tables\present_condition.csv
Input_Files/lookup_tables\record_source.csv
Input_Files/lookup_tables\reporting_method.csv
Input_Files/lookup_tables\sex.csv
Input_Files/lookup_tables\species.csv
Input_Files/lookup_tables\who_obtained.csv
age
band_status
band_type
bird_status
coordinates_precision
country_state
event_type
extra_info
how_obtained
inexact_dates
present_condition
record_source
reporting_method
sex
species
who_obtained


In [6]:
#'country_state' not needed as it is already captured in ISO format in the primary dataset
# inexact date also not needed as event_date is provided along with day, month, year -- more on date standardization later

# note that reference datasets are not uniform in ontology/header nomenclature, so a function-based approach may produce errors
# a similar approach will be used for each reference table, with minor variations:
# 1. dataframe will be created from the reference table and visualized to understand the context of the data
# 2. for each each field deemed valuable to be added as metadata to the primary table, a dictionary will be created
#    using the CODE as the key and the target field to be added as the value
# 3. the map functionality in pandas will be used to create a new column in the primary dataset containing the new data.
#    also note some data type transformations/whitespace  are necessary

# now iterating through each of the reference table, starting with age
# note a function-based approach would be erronious as ontology/column header nomenclature in reference tables is not uniform 
age_df = d['age']
age_df['AGE_CODE'] = age_df['AGE_CODE'].astype(str)
age_df.head()

,,AGE_CODE,AGE_ALPHA_CODE,AGE_DESCRIPTION,AGE_EXPLAIN_TEXT
0,1,0,U,Unknown,A bird that cannot be placed in any classes be...
1,2,1,AHY,After Hatching Year,A bird known to have hatched before the calend...
2,3,2,HY,Hatching Year,A bird capable of sustained flight and known t...
3,4,3,J,Juvenile,"OBSOLETE, old code used for nestling or recent..."
4,5,4,L,Local,A nestling or young bird incapable of sustaine...


In [7]:
# can use 'age_description' column to enrich the primary dataset
age_lookup = pd.Series(age_df.AGE_DESCRIPTION.values,index=age_df.AGE_CODE).to_dict()
for key, value in age_lookup.items():
    key = str(key)

# enrich the working dataframe 
working_df['AGE'] = working_df['AGE_CODE'].map(age_lookup)  
working_df.head()

,BAND,ORIGINAL_BAND,OTHER_BANDS,EVENT_TYPE,EVENT_DATE,EVENT_DAY,EVENT_MONTH,EVENT_YEAR,ISO_COUNTRY,ISO_SUBDIVISION,LAT_DD,LON_DD,COORDINATES_PRECISION_CODE,BAND_TYPE_CODE,SPECIES_NAME,SPECIES_ID,BIRD_STATUS,EXTRA_INFO_CODE,AGE_CODE,SEX_CODE,PERMIT,BAND_STATUS_CODE,HOW_OBTAINED_CODE,WHO_OBTAINED_CODE,REPORTING_METHOD_CODE,PRESENT_CONDITION_CODE,MIN_AGE_AT_ENC,RECORD_SOURCE,Column1,AGE
0,B07104722502,B07104722502,NaN,B,6/13/2019,13,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3,1,4,0,P6726807,0,nan,nan,nan,nan,nan,B,NaN,Local
1,B07104722582,B07104722582,NaN,B,6/21/2019,21,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3,1,4,0,P6726807,0,nan,nan,nan,nan,nan,B,NaN,Local
2,B07234100054,B07234100054,NaN,B,8/2/2016,2,8,2016,US,US-ND,46.75000,-100.75000,10,41,Piping Plover,2770,3,69,4,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,Local
3,B07234100061,B07234100061,NaN,B,6/2/2017,2,6,2017,US,US-ND,47.75000,-102.41667,10,41,Piping Plover,2770,3,25,1,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,After Hatching Year
4,B07234100070,B07234100070,NaN,B,5/30/2017,30,5,2017,US,US-ND,47.91667,-102.41667,10,41,Piping Plover,2770,3,25,1,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,After Hatching Year


In [8]:
# now band_status
band_status_df = d['band_status']
band_status_df.head()

,,BAND_STATUS_CODE,BAND_STATUS_DESCRIPTION
0,1,0,Original band
1,2,1,Replacement Band
2,3,2,Additional Band
3,4,4,Bands identified as multiple when encounter wa...
4,5,5,Band lost


In [9]:
band_status_lookup = pd.Series(band_status_df.BAND_STATUS_DESCRIPTION.values,index=band_status_df.BAND_STATUS_CODE).to_dict()
for key, value in band_status_lookup.items():
    key = str(key)

# enrich the working dataframe 
working_df['BAND_STATUS'] = working_df['BAND_STATUS_CODE'].map(band_status_lookup)  
working_df.head()

,BAND,ORIGINAL_BAND,OTHER_BANDS,EVENT_TYPE,EVENT_DATE,EVENT_DAY,EVENT_MONTH,EVENT_YEAR,ISO_COUNTRY,ISO_SUBDIVISION,LAT_DD,LON_DD,COORDINATES_PRECISION_CODE,BAND_TYPE_CODE,SPECIES_NAME,SPECIES_ID,BIRD_STATUS,EXTRA_INFO_CODE,AGE_CODE,SEX_CODE,PERMIT,BAND_STATUS_CODE,HOW_OBTAINED_CODE,WHO_OBTAINED_CODE,REPORTING_METHOD_CODE,PRESENT_CONDITION_CODE,MIN_AGE_AT_ENC,RECORD_SOURCE,Column1,AGE,BAND_STATUS
0,B07104722502,B07104722502,NaN,B,6/13/2019,13,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3,1,4,0,P6726807,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band
1,B07104722582,B07104722582,NaN,B,6/21/2019,21,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3,1,4,0,P6726807,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band
2,B07234100054,B07234100054,NaN,B,8/2/2016,2,8,2016,US,US-ND,46.75000,-100.75000,10,41,Piping Plover,2770,3,69,4,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band
3,B07234100061,B07234100061,NaN,B,6/2/2017,2,6,2017,US,US-ND,47.75000,-102.41667,10,41,Piping Plover,2770,3,25,1,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,After Hatching Year,Original band
4,B07234100070,B07234100070,NaN,B,5/30/2017,30,5,2017,US,US-ND,47.91667,-102.41667,10,41,Piping Plover,2770,3,25,1,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,After Hatching Year,Original band


In [10]:
# now band type
band_type_df = d['band_type']
band_type_df.head()

,,BAND_TYPE_CODE,BAND_TYPE_DESCRIPTION,BAND_CLOSURE
0,1,41,aluminum\butt-end (toll-free /web address),BE
1,2,42,aluminum\lock-on (toll-free /web address),LO
2,3,46,aluminum\rivet (toll-free /web address),RI
3,4,47,aluminum\short (toll-free /web address),SB
4,5,81,stainless steel (web address),BE


In [11]:
# for band type there are two fields that should be extracted and captured in the primary dataset, description and closure type
band_type_desc_lookup = pd.Series(band_type_df.BAND_TYPE_DESCRIPTION.values,index=band_type_df.BAND_TYPE_CODE).to_dict()
for key, value in band_status_lookup.items():
    key = str(key)
    
band_type_clos_lookup = pd.Series(band_type_df.BAND_CLOSURE.values,index=band_type_df.BAND_TYPE_CODE).to_dict()
for key, value in band_status_lookup.items():
    key = str(key)

# enrich the working dataframe 
working_df['BAND_TYPE'] = working_df['BAND_TYPE_CODE'].map(band_type_desc_lookup)
working_df['BAND_CLOSURE_TYPE'] = working_df['BAND_TYPE_CODE'].map(band_type_clos_lookup)  
working_df.head()

,BAND,ORIGINAL_BAND,OTHER_BANDS,EVENT_TYPE,EVENT_DATE,EVENT_DAY,EVENT_MONTH,EVENT_YEAR,ISO_COUNTRY,ISO_SUBDIVISION,LAT_DD,LON_DD,COORDINATES_PRECISION_CODE,BAND_TYPE_CODE,SPECIES_NAME,SPECIES_ID,BIRD_STATUS,EXTRA_INFO_CODE,AGE_CODE,SEX_CODE,PERMIT,BAND_STATUS_CODE,HOW_OBTAINED_CODE,WHO_OBTAINED_CODE,REPORTING_METHOD_CODE,PRESENT_CONDITION_CODE,MIN_AGE_AT_ENC,RECORD_SOURCE,Column1,AGE,BAND_STATUS,BAND_TYPE,BAND_CLOSURE_TYPE
0,B07104722502,B07104722502,NaN,B,6/13/2019,13,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3,1,4,0,P6726807,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,Aluminum butt-end web address,BE
1,B07104722582,B07104722582,NaN,B,6/21/2019,21,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3,1,4,0,P6726807,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,Aluminum butt-end web address,BE
2,B07234100054,B07234100054,NaN,B,8/2/2016,2,8,2016,US,US-ND,46.75000,-100.75000,10,41,Piping Plover,2770,3,69,4,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,aluminum\butt-end (toll-free /web address),BE
3,B07234100061,B07234100061,NaN,B,6/2/2017,2,6,2017,US,US-ND,47.75000,-102.41667,10,41,Piping Plover,2770,3,25,1,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,After Hatching Year,Original band,aluminum\butt-end (toll-free /web address),BE
4,B07234100070,B07234100070,NaN,B,5/30/2017,30,5,2017,US,US-ND,47.91667,-102.41667,10,41,Piping Plover,2770,3,25,1,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,After Hatching Year,Original band,aluminum\butt-end (toll-free /web address),BE


In [12]:
#now bird status
bird_status_df = d['bird_status']
bird_status_df.head()

,,BIRD_STATUS,BIRD_STATUS_DESCRIPTION,BIRD_STATUS_EXPLAIN_TEXT
0,1,3,Normal wild bird.,"Normal, wild bird: released in same 10-minute ..."
1,2,4,"Hand-reared, game-farm or hacked bird.",Hand-reared or hacked: raised in captivity fro...
2,3,6,Obsolete - Experimental bird.,Obsolete. Formerly used with experimental bird...
3,4,9,Obsolete - Dog caught bird.,Obsolete. Formerly used with dog-caught birds ...
4,5,2,Transported.,"Transported to a different 10-minute block, bu..."


In [13]:
bird_status_lookup = pd.Series(bird_status_df.BIRD_STATUS_DESCRIPTION.values,index=bird_status_df.BIRD_STATUS).to_dict()
for key, value in bird_status_lookup.items():
    key = str(key)

# enrich the working dataframe - note lack of word 'code' in working dataset column name "bird_status" rather than "bird_status_code"
working_df['BIRD_STATUS_DESCRIPTION'] = working_df['BIRD_STATUS'].map(bird_status_lookup)  
working_df.head()

,BAND,ORIGINAL_BAND,OTHER_BANDS,EVENT_TYPE,EVENT_DATE,EVENT_DAY,EVENT_MONTH,EVENT_YEAR,ISO_COUNTRY,ISO_SUBDIVISION,LAT_DD,LON_DD,COORDINATES_PRECISION_CODE,BAND_TYPE_CODE,SPECIES_NAME,SPECIES_ID,BIRD_STATUS,EXTRA_INFO_CODE,AGE_CODE,SEX_CODE,PERMIT,BAND_STATUS_CODE,HOW_OBTAINED_CODE,WHO_OBTAINED_CODE,REPORTING_METHOD_CODE,PRESENT_CONDITION_CODE,MIN_AGE_AT_ENC,RECORD_SOURCE,Column1,AGE,BAND_STATUS,BAND_TYPE,BAND_CLOSURE_TYPE,BIRD_STATUS_DESCRIPTION
0,B07104722502,B07104722502,NaN,B,6/13/2019,13,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3,1,4,0,P6726807,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,Aluminum butt-end web address,BE,Normal wild bird.
1,B07104722582,B07104722582,NaN,B,6/21/2019,21,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3,1,4,0,P6726807,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,Aluminum butt-end web address,BE,Normal wild bird.
2,B07234100054,B07234100054,NaN,B,8/2/2016,2,8,2016,US,US-ND,46.75000,-100.75000,10,41,Piping Plover,2770,3,69,4,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.
3,B07234100061,B07234100061,NaN,B,6/2/2017,2,6,2017,US,US-ND,47.75000,-102.41667,10,41,Piping Plover,2770,3,25,1,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,After Hatching Year,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.
4,B07234100070,B07234100070,NaN,B,5/30/2017,30,5,2017,US,US-ND,47.91667,-102.41667,10,41,Piping Plover,2770,3,25,1,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,After Hatching Year,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.


In [14]:
# now coordinates_precision
coordinates_precision_df = d['coordinates_precision']
coordinates_precision_df.head()

,,COORDINATES_PRECISION_CODE,COORDINATES_PRECISION_DESC
0,1,10,10 minute block
1,2,1,1 minute block
2,3,0,exact
3,4,60,1 degree block
4,5,7,County


In [15]:
coordinates_precision_lookup = pd.Series(coordinates_precision_df.COORDINATES_PRECISION_DESC.values,
                                         index=coordinates_precision_df.COORDINATES_PRECISION_CODE).to_dict()

for key, value in coordinates_precision_lookup.items():
    key = int(key)

# enrich the working dataframe
working_df['COORDINATES_PRECISION_CODE'] = working_df['COORDINATES_PRECISION_CODE'].astype(int)
working_df['COORDINATES_PRECISION'] = working_df['COORDINATES_PRECISION_CODE'].map(coordinates_precision_lookup)  
working_df.head()

,BAND,ORIGINAL_BAND,OTHER_BANDS,EVENT_TYPE,EVENT_DATE,EVENT_DAY,EVENT_MONTH,EVENT_YEAR,ISO_COUNTRY,ISO_SUBDIVISION,LAT_DD,LON_DD,COORDINATES_PRECISION_CODE,BAND_TYPE_CODE,SPECIES_NAME,SPECIES_ID,BIRD_STATUS,EXTRA_INFO_CODE,AGE_CODE,SEX_CODE,PERMIT,BAND_STATUS_CODE,HOW_OBTAINED_CODE,WHO_OBTAINED_CODE,REPORTING_METHOD_CODE,PRESENT_CONDITION_CODE,MIN_AGE_AT_ENC,RECORD_SOURCE,Column1,AGE,BAND_STATUS,BAND_TYPE,BAND_CLOSURE_TYPE,BIRD_STATUS_DESCRIPTION,COORDINATES_PRECISION
0,B07104722502,B07104722502,NaN,B,6/13/2019,13,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3,1,4,0,P6726807,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,Aluminum butt-end web address,BE,Normal wild bird.,exact
1,B07104722582,B07104722582,NaN,B,6/21/2019,21,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3,1,4,0,P6726807,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,Aluminum butt-end web address,BE,Normal wild bird.,exact
2,B07234100054,B07234100054,NaN,B,8/2/2016,2,8,2016,US,US-ND,46.75000,-100.75000,10,41,Piping Plover,2770,3,69,4,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block
3,B07234100061,B07234100061,NaN,B,6/2/2017,2,6,2017,US,US-ND,47.75000,-102.41667,10,41,Piping Plover,2770,3,25,1,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,After Hatching Year,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block
4,B07234100070,B07234100070,NaN,B,5/30/2017,30,5,2017,US,US-ND,47.91667,-102.41667,10,41,Piping Plover,2770,3,25,1,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,After Hatching Year,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block


In [16]:
# now event type
event_type_df = d['event_type']
event_type_df.head()

,,EVENT_TYPE,EVENT_TYPE_DESC
0,1,B,Banding
1,2,E,Encounter


In [17]:
event_type_lookup = pd.Series(event_type_df.EVENT_TYPE_DESC.values,index=event_type_df.EVENT_TYPE).to_dict()

for key, value in event_type_lookup.items():
    key = str(key)

# enrich the working dataframe 
working_df['EVENT_TYPE_DESCRIPTION'] = working_df['EVENT_TYPE'].map(event_type_lookup)
working_df.head()

,BAND,ORIGINAL_BAND,OTHER_BANDS,EVENT_TYPE,EVENT_DATE,EVENT_DAY,EVENT_MONTH,EVENT_YEAR,ISO_COUNTRY,ISO_SUBDIVISION,LAT_DD,LON_DD,COORDINATES_PRECISION_CODE,BAND_TYPE_CODE,SPECIES_NAME,SPECIES_ID,BIRD_STATUS,EXTRA_INFO_CODE,AGE_CODE,SEX_CODE,PERMIT,BAND_STATUS_CODE,HOW_OBTAINED_CODE,WHO_OBTAINED_CODE,REPORTING_METHOD_CODE,PRESENT_CONDITION_CODE,MIN_AGE_AT_ENC,RECORD_SOURCE,Column1,AGE,BAND_STATUS,BAND_TYPE,BAND_CLOSURE_TYPE,BIRD_STATUS_DESCRIPTION,COORDINATES_PRECISION,EVENT_TYPE_DESCRIPTION
0,B07104722502,B07104722502,NaN,B,6/13/2019,13,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3,1,4,0,P6726807,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,Aluminum butt-end web address,BE,Normal wild bird.,exact,Banding
1,B07104722582,B07104722582,NaN,B,6/21/2019,21,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3,1,4,0,P6726807,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,Aluminum butt-end web address,BE,Normal wild bird.,exact,Banding
2,B07234100054,B07234100054,NaN,B,8/2/2016,2,8,2016,US,US-ND,46.75000,-100.75000,10,41,Piping Plover,2770,3,69,4,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block,Banding
3,B07234100061,B07234100061,NaN,B,6/2/2017,2,6,2017,US,US-ND,47.75000,-102.41667,10,41,Piping Plover,2770,3,25,1,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,After Hatching Year,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block,Banding
4,B07234100070,B07234100070,NaN,B,5/30/2017,30,5,2017,US,US-ND,47.91667,-102.41667,10,41,Piping Plover,2770,3,25,1,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,After Hatching Year,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block,Banding


In [18]:
# now extra info
extra_info_df = d['extra_info']
extra_info_df.head()

,,EXTRA_INFO_CODE,EXTRA_INFO_CODE_DESCRIPTION,EXTRA_INFO_LONG_DESCRIPTION
0,1,18,Blood sample taken.,Blood sample taken (contact the appropriate Bi...
1,2,20,Fostered or cross-fostered into wild nests.,Fostered or cross-fostered into wild nests.
2,3,21,"Fostered or cross-fostered into wild nests, pl...","Fostered or cross-fostered into wild nests, pl..."
3,4,29,"Miscellaneous band, Federal band, plus auxilia...","Miscellaneous band (see 06), Federal band, plu..."
4,5,33,"Taken from an artificial nest structure (eg, n...","Taken from an artificial nest structure (e.g.,..."


In [19]:
extra_info_lookup = pd.Series(extra_info_df.EXTRA_INFO_CODE_DESCRIPTION.values,index=extra_info_df.EXTRA_INFO_CODE).to_dict()

for key, value in extra_info_lookup.items():
    key = str(key)

# enrich the working dataframe 
working_df['EXTRA_INFO'] = working_df['EXTRA_INFO_CODE'].map(extra_info_lookup)
working_df.head()

,BAND,ORIGINAL_BAND,OTHER_BANDS,EVENT_TYPE,EVENT_DATE,EVENT_DAY,EVENT_MONTH,EVENT_YEAR,ISO_COUNTRY,ISO_SUBDIVISION,LAT_DD,LON_DD,COORDINATES_PRECISION_CODE,BAND_TYPE_CODE,SPECIES_NAME,SPECIES_ID,BIRD_STATUS,EXTRA_INFO_CODE,AGE_CODE,SEX_CODE,PERMIT,BAND_STATUS_CODE,HOW_OBTAINED_CODE,WHO_OBTAINED_CODE,REPORTING_METHOD_CODE,PRESENT_CONDITION_CODE,MIN_AGE_AT_ENC,RECORD_SOURCE,Column1,AGE,BAND_STATUS,BAND_TYPE,BAND_CLOSURE_TYPE,BIRD_STATUS_DESCRIPTION,COORDINATES_PRECISION,EVENT_TYPE_DESCRIPTION,EXTRA_INFO
0,B07104722502,B07104722502,NaN,B,6/13/2019,13,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3,1,4,0,P6726807,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,Aluminum butt-end web address,BE,Normal wild bird.,exact,Banding,NaN
1,B07104722582,B07104722582,NaN,B,6/21/2019,21,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3,1,4,0,P6726807,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,Aluminum butt-end web address,BE,Normal wild bird.,exact,Banding,NaN
2,B07234100054,B07234100054,NaN,B,8/2/2016,2,8,2016,US,US-ND,46.75000,-100.75000,10,41,Piping Plover,2770,3,69,4,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block,Banding,"Flag, streamer, or tab on leg."
3,B07234100061,B07234100061,NaN,B,6/2/2017,2,6,2017,US,US-ND,47.75000,-102.41667,10,41,Piping Plover,2770,3,25,1,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,After Hatching Year,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block,Banding,Two or more types of auxiliary markers.
4,B07234100070,B07234100070,NaN,B,5/30/2017,30,5,2017,US,US-ND,47.91667,-102.41667,10,41,Piping Plover,2770,3,25,1,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,After Hatching Year,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block,Banding,Two or more types of auxiliary markers.


In [20]:
# now how_obtained
how_obtained_df = d['how_obtained']
how_obtained_df['HOW_OBTAINED_CODE'] = how_obtained_df['HOW_OBTAINED_CODE'].astype(str)
how_obtained_df.head()

,,HOW_OBTAINED_CODE,HOW_OBTAINED_DESCRIPTION,HOW_OBTAINED_CODE_STATUS,HOW_OBTAINED_SEE_ALSO_TEXT
0,1,87,Sight record in a different 10-minute block fr...,discontinued,Code discontinued.
1,2,88,Found nesting in a different 10-minute block f...,discontinued,Code discontinued.
2,3,89,Previously banded bird trapped and released du...,discontinued,Previously banded bird trapped and released du...
3,4,96,Band only,discontinued,Code discontinued.
4,5,97,Miscellaneous. Method of recovery not covered ...,current,Miscellaneous. Method of recovery not covered ...


In [21]:
how_obtained_lookup = pd.Series(how_obtained_df.HOW_OBTAINED_DESCRIPTION.values,
                                index=how_obtained_df.HOW_OBTAINED_CODE).to_dict()

for key, value in how_obtained_lookup.items():
    key = str(key)
    
# enrich the working dataframe
working_df['HOW_OBTAINED_CODE'] = working_df['HOW_OBTAINED_CODE'].astype(int, errors='ignore')
working_df['HOW_OBTAINED'] = working_df['HOW_OBTAINED_CODE'].map(how_obtained_lookup)  
working_df.head()

,BAND,ORIGINAL_BAND,OTHER_BANDS,EVENT_TYPE,EVENT_DATE,EVENT_DAY,EVENT_MONTH,EVENT_YEAR,ISO_COUNTRY,ISO_SUBDIVISION,LAT_DD,LON_DD,COORDINATES_PRECISION_CODE,BAND_TYPE_CODE,SPECIES_NAME,SPECIES_ID,BIRD_STATUS,EXTRA_INFO_CODE,AGE_CODE,SEX_CODE,PERMIT,BAND_STATUS_CODE,HOW_OBTAINED_CODE,WHO_OBTAINED_CODE,REPORTING_METHOD_CODE,PRESENT_CONDITION_CODE,MIN_AGE_AT_ENC,RECORD_SOURCE,Column1,AGE,BAND_STATUS,BAND_TYPE,BAND_CLOSURE_TYPE,BIRD_STATUS_DESCRIPTION,COORDINATES_PRECISION,EVENT_TYPE_DESCRIPTION,EXTRA_INFO,HOW_OBTAINED
0,B07104722502,B07104722502,NaN,B,6/13/2019,13,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3,1,4,0,P6726807,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,Aluminum butt-end web address,BE,Normal wild bird.,exact,Banding,NaN,NaN
1,B07104722582,B07104722582,NaN,B,6/21/2019,21,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3,1,4,0,P6726807,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,Aluminum butt-end web address,BE,Normal wild bird.,exact,Banding,NaN,NaN
2,B07234100054,B07234100054,NaN,B,8/2/2016,2,8,2016,US,US-ND,46.75000,-100.75000,10,41,Piping Plover,2770,3,69,4,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block,Banding,"Flag, streamer, or tab on leg.",NaN
3,B07234100061,B07234100061,NaN,B,6/2/2017,2,6,2017,US,US-ND,47.75000,-102.41667,10,41,Piping Plover,2770,3,25,1,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,After Hatching Year,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block,Banding,Two or more types of auxiliary markers.,NaN
4,B07234100070,B07234100070,NaN,B,5/30/2017,30,5,2017,US,US-ND,47.91667,-102.41667,10,41,Piping Plover,2770,3,25,1,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,After Hatching Year,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block,Banding,Two or more types of auxiliary markers.,NaN


In [22]:
# QA check to ensure all codes were interpretted correctly and no values were missed
n = len(pd.unique(working_df['HOW_OBTAINED']))
n1 = len(pd.unique(working_df['HOW_OBTAINED_CODE']))
list_unique_all = pd.unique(working_df['HOW_OBTAINED']) 
list_unique_all_codes = pd.unique(working_df['HOW_OBTAINED_CODE']) 
print(n)
print(list_unique_all)
print(n1)
print(list_unique_all_codes)

12
[nan
 'Saw or photographed neck collar, color band, or other marker (not federal band) while bird was free'
 'Previously banded bird trapped and released during banding operations'
 'Found dead bird.' 'Saw or photographed federal band while bird was free'
 'Captured for Scientific Purposes (not collected). Status changed.'
 'Caught by or due to: entanglement in fishing gear'
 'Found dead or caught due to disease'
 'Caught due to striking or being struck by: motor vehicle.' 'Shot.'
 'Caught due to: injury' 'Caught by hand.']
12
['nan' '29' '66' '0' '52' '53' '26' '61' '14' '1' '3' '28']


In [23]:
# now present condition
present_condition_df = d['present_condition']
present_condition_df['PRESENT_CONDITION_CODE'] = present_condition_df['PRESENT_CONDITION_CODE'].astype(str)
present_condition_df.head()

,,PRESENT_CONDITION_CODE,PRESENT_CONDITION_BIRD,PRESENT_CONDITION_BAND
0,1,0,UNKNOWN,UNKNOWN
1,2,1,UNKNOWN,LEFT ON BIRD
2,3,2,UNKNOWN,REMOVED
3,4,3,DEAD,UNKNOWN
4,5,4,DEAD,LEFT ON BIRD


In [24]:
# extracting two fields - 'PRESENT_CONDITION_BIRD' and 'PRESENT_CONDITION_BAND'
present_condition_bird_lookup = pd.Series(present_condition_df.PRESENT_CONDITION_BIRD.values,
                                          index=present_condition_df.PRESENT_CONDITION_CODE).to_dict()

for key, value in present_condition_bird_lookup.items():
    key = str(key)
    
present_condition_band_lookup = pd.Series(present_condition_df.PRESENT_CONDITION_BAND.values,
                                          index=present_condition_df.PRESENT_CONDITION_CODE).to_dict()

for key, value in present_condition_band_lookup.items():
    key = str(key)

# enrich the working dataframe 
working_df['PRESENT_CONDITION_BIRD'] = working_df['PRESENT_CONDITION_CODE'].map(present_condition_bird_lookup)
working_df['PRESENT_CONDITION_BAND'] = working_df['PRESENT_CONDITION_CODE'].map(present_condition_band_lookup)
working_df.head()


,BAND,ORIGINAL_BAND,OTHER_BANDS,EVENT_TYPE,EVENT_DATE,EVENT_DAY,EVENT_MONTH,EVENT_YEAR,ISO_COUNTRY,ISO_SUBDIVISION,LAT_DD,LON_DD,COORDINATES_PRECISION_CODE,BAND_TYPE_CODE,SPECIES_NAME,SPECIES_ID,BIRD_STATUS,EXTRA_INFO_CODE,AGE_CODE,SEX_CODE,PERMIT,BAND_STATUS_CODE,HOW_OBTAINED_CODE,WHO_OBTAINED_CODE,REPORTING_METHOD_CODE,PRESENT_CONDITION_CODE,MIN_AGE_AT_ENC,RECORD_SOURCE,Column1,AGE,BAND_STATUS,BAND_TYPE,BAND_CLOSURE_TYPE,BIRD_STATUS_DESCRIPTION,COORDINATES_PRECISION,EVENT_TYPE_DESCRIPTION,EXTRA_INFO,HOW_OBTAINED,PRESENT_CONDITION_BIRD,PRESENT_CONDITION_BAND
0,B07104722502,B07104722502,NaN,B,6/13/2019,13,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3,1,4,0,P6726807,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,Aluminum butt-end web address,BE,Normal wild bird.,exact,Banding,NaN,NaN,NaN,NaN
1,B07104722582,B07104722582,NaN,B,6/21/2019,21,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3,1,4,0,P6726807,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,Aluminum butt-end web address,BE,Normal wild bird.,exact,Banding,NaN,NaN,NaN,NaN
2,B07234100054,B07234100054,NaN,B,8/2/2016,2,8,2016,US,US-ND,46.75000,-100.75000,10,41,Piping Plover,2770,3,69,4,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block,Banding,"Flag, streamer, or tab on leg.",NaN,NaN,NaN
3,B07234100061,B07234100061,NaN,B,6/2/2017,2,6,2017,US,US-ND,47.75000,-102.41667,10,41,Piping Plover,2770,3,25,1,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,After Hatching Year,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block,Banding,Two or more types of auxiliary markers.,NaN,NaN,NaN
4,B07234100070,B07234100070,NaN,B,5/30/2017,30,5,2017,US,US-ND,47.91667,-102.41667,10,41,Piping Plover,2770,3,25,1,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,After Hatching Year,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block,Banding,Two or more types of auxiliary markers.,NaN,NaN,NaN


In [25]:
# quick QA check to make sure the last two integrations worked since all head values are NaN
n = len(pd.unique(working_df['PRESENT_CONDITION_BIRD']))
n2 = len(pd.unique(working_df['PRESENT_CONDITION_BAND']))
n3 = len(pd.unique(working_df['PRESENT_CONDITION_CODE']))
list_unique_all_bird = pd.unique(working_df['PRESENT_CONDITION_BIRD']) 
list_unique_all_band = pd.unique(working_df['PRESENT_CONDITION_BAND'])
print(n)
print(n2)
print(n3)
print(list_unique_all_bird)
print(list_unique_all_band)

5
4
9
[nan 'ALIVE - UNKNOWN' 'ALIVE - RELEASED' 'DEAD' 'ALIVE - IN CAPTIVITY']
[nan 'LEFT ON BIRD' 'REMOVED' 'UNKNOWN']


In [26]:
# now record_source
record_source_df = d['record_source']
record_source_df.head()

,,EVENT_TYPE,EVENT_TYPE_DESC
0,1,B,Banding DB
1,2,E,Encounter DB
2,3,R,Recapture DB


In [27]:
record_source_lookup = pd.Series(record_source_df.EVENT_TYPE_DESC.values,index=record_source_df.EVENT_TYPE).to_dict()

for key, value in record_source_lookup.items():
    key = str(key)

# enrich the working dataframe 
working_df['RECORD_SOURCE_DESCRIPTION'] = working_df['RECORD_SOURCE'].map(record_source_lookup)
working_df.head()

,BAND,ORIGINAL_BAND,OTHER_BANDS,EVENT_TYPE,EVENT_DATE,EVENT_DAY,EVENT_MONTH,EVENT_YEAR,ISO_COUNTRY,ISO_SUBDIVISION,LAT_DD,LON_DD,COORDINATES_PRECISION_CODE,BAND_TYPE_CODE,SPECIES_NAME,SPECIES_ID,BIRD_STATUS,EXTRA_INFO_CODE,AGE_CODE,SEX_CODE,PERMIT,BAND_STATUS_CODE,HOW_OBTAINED_CODE,WHO_OBTAINED_CODE,REPORTING_METHOD_CODE,PRESENT_CONDITION_CODE,MIN_AGE_AT_ENC,RECORD_SOURCE,Column1,AGE,BAND_STATUS,BAND_TYPE,BAND_CLOSURE_TYPE,BIRD_STATUS_DESCRIPTION,COORDINATES_PRECISION,EVENT_TYPE_DESCRIPTION,EXTRA_INFO,HOW_OBTAINED,PRESENT_CONDITION_BIRD,PRESENT_CONDITION_BAND,RECORD_SOURCE_DESCRIPTION
0,B07104722502,B07104722502,NaN,B,6/13/2019,13,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3,1,4,0,P6726807,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,Aluminum butt-end web address,BE,Normal wild bird.,exact,Banding,NaN,NaN,NaN,NaN,Banding DB
1,B07104722582,B07104722582,NaN,B,6/21/2019,21,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3,1,4,0,P6726807,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,Aluminum butt-end web address,BE,Normal wild bird.,exact,Banding,NaN,NaN,NaN,NaN,Banding DB
2,B07234100054,B07234100054,NaN,B,8/2/2016,2,8,2016,US,US-ND,46.75000,-100.75000,10,41,Piping Plover,2770,3,69,4,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block,Banding,"Flag, streamer, or tab on leg.",NaN,NaN,NaN,Banding DB
3,B07234100061,B07234100061,NaN,B,6/2/2017,2,6,2017,US,US-ND,47.75000,-102.41667,10,41,Piping Plover,2770,3,25,1,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,After Hatching Year,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block,Banding,Two or more types of auxiliary markers.,NaN,NaN,NaN,Banding DB
4,B07234100070,B07234100070,NaN,B,5/30/2017,30,5,2017,US,US-ND,47.91667,-102.41667,10,41,Piping Plover,2770,3,25,1,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,After Hatching Year,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block,Banding,Two or more types of auxiliary markers.,NaN,NaN,NaN,Banding DB


In [28]:
# now reporting_method
reporting_method_df = d['reporting_method']
reporting_method_df.head()

,,REPORTING_METHOD_CODE,REPORTING_METHOD_DESC
0,1,0,Unknown
1,2,1,The band number was reported by or as a result...
2,3,2,The band number was reported by or as a result...
3,4,6,Reported on the 1-800 telephone number. This c...
4,5,7,Reported on the Internet. This code was added ...


In [29]:
reporting_method_lookup = pd.Series(reporting_method_df.REPORTING_METHOD_DESC.values,
                                    index=reporting_method_df.REPORTING_METHOD_CODE.astype(str)).to_dict()

# enrich the working dataframe 
working_df['REPORTING_METHOD'] = working_df['REPORTING_METHOD_CODE'].map(reporting_method_lookup)
working_df.head()

,BAND,ORIGINAL_BAND,OTHER_BANDS,EVENT_TYPE,EVENT_DATE,EVENT_DAY,EVENT_MONTH,EVENT_YEAR,ISO_COUNTRY,ISO_SUBDIVISION,LAT_DD,LON_DD,COORDINATES_PRECISION_CODE,BAND_TYPE_CODE,SPECIES_NAME,SPECIES_ID,BIRD_STATUS,EXTRA_INFO_CODE,AGE_CODE,SEX_CODE,PERMIT,BAND_STATUS_CODE,HOW_OBTAINED_CODE,WHO_OBTAINED_CODE,REPORTING_METHOD_CODE,PRESENT_CONDITION_CODE,MIN_AGE_AT_ENC,RECORD_SOURCE,Column1,AGE,BAND_STATUS,BAND_TYPE,BAND_CLOSURE_TYPE,BIRD_STATUS_DESCRIPTION,COORDINATES_PRECISION,EVENT_TYPE_DESCRIPTION,EXTRA_INFO,HOW_OBTAINED,PRESENT_CONDITION_BIRD,PRESENT_CONDITION_BAND,RECORD_SOURCE_DESCRIPTION,REPORTING_METHOD
0,B07104722502,B07104722502,NaN,B,6/13/2019,13,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3,1,4,0,P6726807,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,Aluminum butt-end web address,BE,Normal wild bird.,exact,Banding,NaN,NaN,NaN,NaN,Banding DB,NaN
1,B07104722582,B07104722582,NaN,B,6/21/2019,21,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3,1,4,0,P6726807,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,Aluminum butt-end web address,BE,Normal wild bird.,exact,Banding,NaN,NaN,NaN,NaN,Banding DB,NaN
2,B07234100054,B07234100054,NaN,B,8/2/2016,2,8,2016,US,US-ND,46.75000,-100.75000,10,41,Piping Plover,2770,3,69,4,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block,Banding,"Flag, streamer, or tab on leg.",NaN,NaN,NaN,Banding DB,NaN
3,B07234100061,B07234100061,NaN,B,6/2/2017,2,6,2017,US,US-ND,47.75000,-102.41667,10,41,Piping Plover,2770,3,25,1,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,After Hatching Year,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block,Banding,Two or more types of auxiliary markers.,NaN,NaN,NaN,Banding DB,NaN
4,B07234100070,B07234100070,NaN,B,5/30/2017,30,5,2017,US,US-ND,47.91667,-102.41667,10,41,Piping Plover,2770,3,25,1,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,After Hatching Year,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block,Banding,Two or more types of auxiliary markers.,NaN,NaN,NaN,Banding DB,NaN


In [30]:
# another QA check since all in head are NaN
n = len(pd.unique(working_df['REPORTING_METHOD']))
n1 = len(pd.unique(working_df['REPORTING_METHOD_CODE']))
list_unique_all_code = pd.unique(working_df['REPORTING_METHOD_CODE']) 
list_unique_all_values = pd.unique(working_df['REPORTING_METHOD'])
print(n)
print(n1)
print(list_unique_all_code)
print(list_unique_all_values)

11
11
['nan' '13' '14' '65' '7' '1' '12' '62' '11' '63' '61']
[nan 'Electronic color marker report'
 'Bandit, Band Manager, or paper banding schedule'
 'Phone, color marker report'
 'Reported on the Internet. This code was added December 1, 1999 to keep track of bands reported on the Internet.'
 'The band number was reported by or as a result of the initiative of the person who found the band, their family, or a personal acquaintance. Why Reported 01 is used if a conservation agent reports a band at the finders request. Since 09/01/1995 - Paper.'
 'Email (other than from color marker reports)' '1-800 report, USGSBBL'
 'Paper, postal mail' '1-800 report, CWSBBO' '1-800 report, call center']


In [31]:
# now sex
sex_df = d['sex']
sex_df.head()

,,SEX_CODE,SEX_DESCRIPTION
0,1,0,Unknown
1,2,4,Male
2,3,5,Female
3,4,6,"Male, sexed upon recapture"
4,5,7,"Female, sexed upon recapture"


In [32]:
sex_lookup = pd.Series(sex_df.SEX_DESCRIPTION.values,index=sex_df.SEX_CODE.astype(str)).to_dict()

# sex_code 6 and 7 are not necessary - should be limited to male, female, and unknown
working_df['SEX_CODE'] = working_df['SEX_CODE'].replace('6', '4')
working_df['SEX_CODE'] = working_df['SEX_CODE'].replace('7', '5')

# enrich the working dataframe 
working_df['SEX'] = working_df['SEX_CODE'].map(sex_lookup)

print(working_df['SEX'].describe())
working_df.head()

count       50022
unique          3
top       Unknown
freq        42231
Name: SEX, dtype: object


,BAND,ORIGINAL_BAND,OTHER_BANDS,EVENT_TYPE,EVENT_DATE,EVENT_DAY,EVENT_MONTH,EVENT_YEAR,ISO_COUNTRY,ISO_SUBDIVISION,LAT_DD,LON_DD,COORDINATES_PRECISION_CODE,BAND_TYPE_CODE,SPECIES_NAME,SPECIES_ID,BIRD_STATUS,EXTRA_INFO_CODE,AGE_CODE,SEX_CODE,PERMIT,BAND_STATUS_CODE,HOW_OBTAINED_CODE,WHO_OBTAINED_CODE,REPORTING_METHOD_CODE,PRESENT_CONDITION_CODE,MIN_AGE_AT_ENC,RECORD_SOURCE,Column1,AGE,BAND_STATUS,BAND_TYPE,BAND_CLOSURE_TYPE,BIRD_STATUS_DESCRIPTION,COORDINATES_PRECISION,EVENT_TYPE_DESCRIPTION,EXTRA_INFO,HOW_OBTAINED,PRESENT_CONDITION_BIRD,PRESENT_CONDITION_BAND,RECORD_SOURCE_DESCRIPTION,REPORTING_METHOD,SEX
0,B07104722502,B07104722502,NaN,B,6/13/2019,13,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3,1,4,0,P6726807,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,Aluminum butt-end web address,BE,Normal wild bird.,exact,Banding,NaN,NaN,NaN,NaN,Banding DB,NaN,Unknown
1,B07104722582,B07104722582,NaN,B,6/21/2019,21,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3,1,4,0,P6726807,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,Aluminum butt-end web address,BE,Normal wild bird.,exact,Banding,NaN,NaN,NaN,NaN,Banding DB,NaN,Unknown
2,B07234100054,B07234100054,NaN,B,8/2/2016,2,8,2016,US,US-ND,46.75000,-100.75000,10,41,Piping Plover,2770,3,69,4,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block,Banding,"Flag, streamer, or tab on leg.",NaN,NaN,NaN,Banding DB,NaN,Unknown
3,B07234100061,B07234100061,NaN,B,6/2/2017,2,6,2017,US,US-ND,47.75000,-102.41667,10,41,Piping Plover,2770,3,25,1,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,After Hatching Year,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block,Banding,Two or more types of auxiliary markers.,NaN,NaN,NaN,Banding DB,NaN,Unknown
4,B07234100070,B07234100070,NaN,B,5/30/2017,30,5,2017,US,US-ND,47.91667,-102.41667,10,41,Piping Plover,2770,3,25,1,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,After Hatching Year,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block,Banding,Two or more types of auxiliary markers.,NaN,NaN,NaN,Banding DB,NaN,Unknown


In [33]:
#species
species_df = d['species']
species_df.head()

,,SPECIES_ID,SPECIES_NAME,ALPHA_CODE,Download,TAXONOMIC_ORDER,SCI_NAME,RECOMENDSIZE,ALLOWABLESIZE,ENDANGERED,RAPTOR,GAMEBIRD
0,208,1690,Lesser Snow Goose,LSGO,1.0,10.0,Anser caerulescens caerulescens,7B,"7B, 7",NaN,NaN,Y
1,209,1691,Blue Goose,BLGO,1.0,11.0,Anser caerulescens,7B,"7B, 7",NaN,NaN,Y
2,210,1695,Snow X Blue Goose Intergrade,SBGI,1.0,12.0,NaN,7B,7B,NaN,NaN,Y
3,211,1698,Blue Greater Snow Goose,BGSG,1.0,13.0,NaN,7B,7B,NaN,NaN,Y
4,212,1699,Greater Snow Goose,GSGO,1.0,14.0,Anser caerulescens atlantica,7B,7B,NaN,NaN,Y


In [34]:
# taking species_name, alpha_code, taxonomic_order, endangered, and allowablesize from the species DF
species_name_lookup = pd.Series(species_df.SPECIES_NAME.values,
                                    index=species_df.SPECIES_ID.astype(str)).to_dict()
species_alpha_lookup = pd.Series(species_df.ALPHA_CODE.values,
                                    index=species_df.SPECIES_ID.astype(str)).to_dict()
species_tax_order_lookup = pd.Series(species_df.TAXONOMIC_ORDER.values,
                                    index=species_df.SPECIES_ID.astype(str)).to_dict()
species_endangered_lookup = pd.Series(species_df.ENDANGERED.values,
                                    index=species_df.SPECIES_ID.astype(str)).to_dict()
species_allowable_size_lookup = pd.Series(species_df.ALLOWABLESIZE.values,
                                    index=species_df.SPECIES_ID.astype(str)).to_dict()

# enrich the working dataframe 
working_df['SPECIES_NAME'] = working_df['SPECIES_ID'].map(species_name_lookup)
working_df['SPECIES_ALPHA_CODE'] = working_df['SPECIES_ID'].map(species_alpha_lookup)
working_df['TAXONOMIC_ORDER'] = working_df['SPECIES_ID'].map(species_tax_order_lookup)
working_df['ENDANGERED'] = working_df['SPECIES_ID'].map(species_endangered_lookup)
working_df['ALLOWABLE_BAND_SIZE'] = working_df['SPECIES_ID'].map(species_allowable_size_lookup)
working_df.head()

,BAND,ORIGINAL_BAND,OTHER_BANDS,EVENT_TYPE,EVENT_DATE,EVENT_DAY,EVENT_MONTH,EVENT_YEAR,ISO_COUNTRY,ISO_SUBDIVISION,LAT_DD,LON_DD,COORDINATES_PRECISION_CODE,BAND_TYPE_CODE,SPECIES_NAME,SPECIES_ID,BIRD_STATUS,EXTRA_INFO_CODE,AGE_CODE,SEX_CODE,PERMIT,BAND_STATUS_CODE,HOW_OBTAINED_CODE,WHO_OBTAINED_CODE,REPORTING_METHOD_CODE,PRESENT_CONDITION_CODE,MIN_AGE_AT_ENC,RECORD_SOURCE,Column1,AGE,BAND_STATUS,BAND_TYPE,BAND_CLOSURE_TYPE,BIRD_STATUS_DESCRIPTION,COORDINATES_PRECISION,EVENT_TYPE_DESCRIPTION,EXTRA_INFO,HOW_OBTAINED,PRESENT_CONDITION_BIRD,PRESENT_CONDITION_BAND,RECORD_SOURCE_DESCRIPTION,REPORTING_METHOD,SEX,SPECIES_ALPHA_CODE,TAXONOMIC_ORDER,ENDANGERED,ALLOWABLE_BAND_SIZE
0,B07104722502,B07104722502,NaN,B,6/13/2019,13,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3,1,4,0,P6726807,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,Aluminum butt-end web address,BE,Normal wild bird.,exact,Banding,NaN,NaN,NaN,NaN,Banding DB,NaN,Unknown,WIPL,302.0,NaN,"1D, 1A, 2"
1,B07104722582,B07104722582,NaN,B,6/21/2019,21,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3,1,4,0,P6726807,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,Aluminum butt-end web address,BE,Normal wild bird.,exact,Banding,NaN,NaN,NaN,NaN,Banding DB,NaN,Unknown,WIPL,302.0,NaN,"1D, 1A, 2"
2,B07234100054,B07234100054,NaN,B,8/2/2016,2,8,2016,US,US-ND,46.75000,-100.75000,10,41,Piping Plover,2770,3,69,4,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block,Banding,"Flag, streamer, or tab on leg.",NaN,NaN,NaN,Banding DB,NaN,Unknown,PIPL,305.0,Y,"1A, 1B"
3,B07234100061,B07234100061,NaN,B,6/2/2017,2,6,2017,US,US-ND,47.75000,-102.41667,10,41,Piping Plover,2770,3,25,1,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,After Hatching Year,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block,Banding,Two or more types of auxiliary markers.,NaN,NaN,NaN,Banding DB,NaN,Unknown,PIPL,305.0,Y,"1A, 1B"
4,B07234100070,B07234100070,NaN,B,5/30/2017,30,5,2017,US,US-ND,47.91667,-102.41667,10,41,Piping Plover,2770,3,25,1,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,After Hatching Year,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block,Banding,Two or more types of auxiliary markers.,NaN,NaN,NaN,Banding DB,NaN,Unknown,PIPL,305.0,Y,"1A, 1B"


In [35]:
#who_obtained
who_obtained_df = d['who_obtained']
who_obtained_df.head()

,,WHO_OBTAINED_CODE,WHO_OBTAINED_DESCRIPTION,WHO_OBTAINED_EXPLAIN_TEXT
0,1,20,MISCELLANEOUS,From a source not included in any other catego...
1,2,21,FINDER,The band was reported either by the person who...
2,3,22,"STATE, PROVINCIAL, OR FEDERAL","This code is used when State, Provincial, or F..."
3,4,23,BIRD BANDERS,This code is used to identify band numbers rep...
4,5,24,U. S. PARTS COLLECTION SURVEY,Each year the U. S. Fish and Wildlife Service ...


In [36]:
who_obtained_lookup = pd.Series(who_obtained_df.WHO_OBTAINED_DESCRIPTION.values,
                                index=who_obtained_df.WHO_OBTAINED_CODE.astype(str)).to_dict()

# enrich the working dataframe 
working_df['WHO_OBTAINED'] = working_df['WHO_OBTAINED_CODE'].map(who_obtained_lookup)
working_df.head()

,BAND,ORIGINAL_BAND,OTHER_BANDS,EVENT_TYPE,EVENT_DATE,EVENT_DAY,EVENT_MONTH,EVENT_YEAR,ISO_COUNTRY,ISO_SUBDIVISION,LAT_DD,LON_DD,COORDINATES_PRECISION_CODE,BAND_TYPE_CODE,SPECIES_NAME,SPECIES_ID,BIRD_STATUS,EXTRA_INFO_CODE,AGE_CODE,SEX_CODE,PERMIT,BAND_STATUS_CODE,HOW_OBTAINED_CODE,WHO_OBTAINED_CODE,REPORTING_METHOD_CODE,PRESENT_CONDITION_CODE,MIN_AGE_AT_ENC,RECORD_SOURCE,Column1,AGE,BAND_STATUS,BAND_TYPE,BAND_CLOSURE_TYPE,BIRD_STATUS_DESCRIPTION,COORDINATES_PRECISION,EVENT_TYPE_DESCRIPTION,EXTRA_INFO,HOW_OBTAINED,PRESENT_CONDITION_BIRD,PRESENT_CONDITION_BAND,RECORD_SOURCE_DESCRIPTION,REPORTING_METHOD,SEX,SPECIES_ALPHA_CODE,TAXONOMIC_ORDER,ENDANGERED,ALLOWABLE_BAND_SIZE,WHO_OBTAINED
0,B07104722502,B07104722502,NaN,B,6/13/2019,13,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3,1,4,0,P6726807,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,Aluminum butt-end web address,BE,Normal wild bird.,exact,Banding,NaN,NaN,NaN,NaN,Banding DB,NaN,Unknown,WIPL,302.0,NaN,"1D, 1A, 2",NaN
1,B07104722582,B07104722582,NaN,B,6/21/2019,21,6,2019,US,US-FL,29.84389,-85.41472,0,W1,Wilson's Plover,2800,3,1,4,0,P6726807,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,Aluminum butt-end web address,BE,Normal wild bird.,exact,Banding,NaN,NaN,NaN,NaN,Banding DB,NaN,Unknown,WIPL,302.0,NaN,"1D, 1A, 2",NaN
2,B07234100054,B07234100054,NaN,B,8/2/2016,2,8,2016,US,US-ND,46.75000,-100.75000,10,41,Piping Plover,2770,3,69,4,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,Local,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block,Banding,"Flag, streamer, or tab on leg.",NaN,NaN,NaN,Banding DB,NaN,Unknown,PIPL,305.0,Y,"1A, 1B",NaN
3,B07234100061,B07234100061,NaN,B,6/2/2017,2,6,2017,US,US-ND,47.75000,-102.41667,10,41,Piping Plover,2770,3,25,1,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,After Hatching Year,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block,Banding,Two or more types of auxiliary markers.,NaN,NaN,NaN,Banding DB,NaN,Unknown,PIPL,305.0,Y,"1A, 1B",NaN
4,B07234100070,B07234100070,NaN,B,5/30/2017,30,5,2017,US,US-ND,47.91667,-102.41667,10,41,Piping Plover,2770,3,25,1,0,P2962009,0,nan,nan,nan,nan,nan,B,NaN,After Hatching Year,Original band,aluminum\butt-end (toll-free /web address),BE,Normal wild bird.,10 minute block,Banding,Two or more types of auxiliary markers.,NaN,NaN,NaN,Banding DB,NaN,Unknown,PIPL,305.0,Y,"1A, 1B",NaN


In [37]:
# another QA check since all in head are NaN
n = len(pd.unique(working_df['WHO_OBTAINED']))
list_unique_all = pd.unique(working_df['WHO_OBTAINED_CODE']) 
print(n)
print(list_unique_all)

5
['nan' '21' '23' '22' '20']


In [38]:
# selection of columns from the current working dataframe to transfer to a consolidated dataframe
# printing complete list of columns
working_df.columns

Index(['BAND', 'ORIGINAL_BAND', 'OTHER_BANDS', 'EVENT_TYPE', 'EVENT_DATE',
       'EVENT_DAY', 'EVENT_MONTH', 'EVENT_YEAR', 'ISO_COUNTRY',
       'ISO_SUBDIVISION', 'LAT_DD', 'LON_DD', 'COORDINATES_PRECISION_CODE',
       'BAND_TYPE_CODE', 'SPECIES_NAME', 'SPECIES_ID', 'BIRD_STATUS',
       'EXTRA_INFO_CODE', 'AGE_CODE', 'SEX_CODE', 'PERMIT', 'BAND_STATUS_CODE',
       'HOW_OBTAINED_CODE', 'WHO_OBTAINED_CODE', 'REPORTING_METHOD_CODE',
       'PRESENT_CONDITION_CODE', 'MIN_AGE_AT_ENC', 'RECORD_SOURCE', 'Column1',
       'AGE', 'BAND_STATUS', 'BAND_TYPE', 'BAND_CLOSURE_TYPE',
       'BIRD_STATUS_DESCRIPTION', 'COORDINATES_PRECISION',
       'EVENT_TYPE_DESCRIPTION', 'EXTRA_INFO', 'HOW_OBTAINED',
       'PRESENT_CONDITION_BIRD', 'PRESENT_CONDITION_BAND',
       'RECORD_SOURCE_DESCRIPTION', 'REPORTING_METHOD', 'SEX',
       'SPECIES_ALPHA_CODE', 'TAXONOMIC_ORDER', 'ENDANGERED',
       'ALLOWABLE_BAND_SIZE', 'WHO_OBTAINED'],
      dtype='object')

In [39]:
working_df_subset = working_df[['BAND', 'ORIGINAL_BAND', 'OTHER_BANDS', 'BAND_TYPE', 'BAND_STATUS', 'PRESENT_CONDITION_BAND',
                               'BAND_CLOSURE_TYPE', 'EVENT_TYPE_DESCRIPTION', 'EVENT_DATE', 'EVENT_DAY',
                               'EVENT_MONTH', 'EVENT_YEAR', 'ISO_COUNTRY', 'ISO_SUBDIVISION', 'LAT_DD', 'LON_DD',
                               'COORDINATES_PRECISION', 'PERMIT', 'HOW_OBTAINED', 'WHO_OBTAINED','REPORTING_METHOD',
                               'SPECIES_ID', 'SPECIES_NAME', 'BIRD_STATUS_DESCRIPTION',
                               'AGE', 'SEX', 'MIN_AGE_AT_ENC', 'SPECIES_ALPHA_CODE', 'TAXONOMIC_ORDER', 'ENDANGERED',
                               'ALLOWABLE_BAND_SIZE', 'PRESENT_CONDITION_BIRD', 'EXTRA_INFO', 'RECORD_SOURCE_DESCRIPTION']].copy()
working_df_subset.head()

,BAND,ORIGINAL_BAND,OTHER_BANDS,BAND_TYPE,BAND_STATUS,PRESENT_CONDITION_BAND,BAND_CLOSURE_TYPE,EVENT_TYPE_DESCRIPTION,EVENT_DATE,EVENT_DAY,EVENT_MONTH,EVENT_YEAR,ISO_COUNTRY,ISO_SUBDIVISION,LAT_DD,LON_DD,COORDINATES_PRECISION,PERMIT,HOW_OBTAINED,WHO_OBTAINED,REPORTING_METHOD,SPECIES_ID,SPECIES_NAME,BIRD_STATUS_DESCRIPTION,AGE,SEX,MIN_AGE_AT_ENC,SPECIES_ALPHA_CODE,TAXONOMIC_ORDER,ENDANGERED,ALLOWABLE_BAND_SIZE,PRESENT_CONDITION_BIRD,EXTRA_INFO,RECORD_SOURCE_DESCRIPTION
0,B07104722502,B07104722502,NaN,Aluminum butt-end web address,Original band,NaN,BE,Banding,6/13/2019,13,6,2019,US,US-FL,29.84389,-85.41472,exact,P6726807,NaN,NaN,NaN,2800,Wilson's Plover,Normal wild bird.,Local,Unknown,nan,WIPL,302.0,NaN,"1D, 1A, 2",NaN,NaN,Banding DB
1,B07104722582,B07104722582,NaN,Aluminum butt-end web address,Original band,NaN,BE,Banding,6/21/2019,21,6,2019,US,US-FL,29.84389,-85.41472,exact,P6726807,NaN,NaN,NaN,2800,Wilson's Plover,Normal wild bird.,Local,Unknown,nan,WIPL,302.0,NaN,"1D, 1A, 2",NaN,NaN,Banding DB
2,B07234100054,B07234100054,NaN,aluminum\butt-end (toll-free /web address),Original band,NaN,BE,Banding,8/2/2016,2,8,2016,US,US-ND,46.75000,-100.75000,10 minute block,P2962009,NaN,NaN,NaN,2770,Piping Plover,Normal wild bird.,Local,Unknown,nan,PIPL,305.0,Y,"1A, 1B",NaN,"Flag, streamer, or tab on leg.",Banding DB
3,B07234100061,B07234100061,NaN,aluminum\butt-end (toll-free /web address),Original band,NaN,BE,Banding,6/2/2017,2,6,2017,US,US-ND,47.75000,-102.41667,10 minute block,P2962009,NaN,NaN,NaN,2770,Piping Plover,Normal wild bird.,After Hatching Year,Unknown,nan,PIPL,305.0,Y,"1A, 1B",NaN,Two or more types of auxiliary markers.,Banding DB
4,B07234100070,B07234100070,NaN,aluminum\butt-end (toll-free /web address),Original band,NaN,BE,Banding,5/30/2017,30,5,2017,US,US-ND,47.91667,-102.41667,10 minute block,P2962009,NaN,NaN,NaN,2770,Piping Plover,Normal wild bird.,After Hatching Year,Unknown,nan,PIPL,305.0,Y,"1A, 1B",NaN,Two or more types of auxiliary markers.,Banding DB


In [40]:
# next step - harmonization of ontologies within species column
# identify inconsistencies in spelling, spacing, format, duplicate errors, and invalid data

In [41]:
# start with species_name
counts = working_df_subset.groupby(['SPECIES_NAME', 'SPECIES_ID'])['BAND'].nunique()
print(counts)

SPECIES_NAME           SPECIES_ID
Black-bellied Plover   2700           2194
Common Tern            700               1
Long-billed Dowitcher  2320              1
Piping Plover          2770          22597
Semipalmated Plover    2740          15454
Snowy Plover           2780           6867
Western Snowy Plover   2781             62
Wilson's Plover        2800           2794
Name: BAND, dtype: int64


In [42]:
# recall the only species used in this dataset were 'Black-bellied Plover', 'Piping Plover', 
# 'Semipalmated Plover', 'Snowy Plover', and 'Wilson's Plover'
# Common Tern and Long-billed Dowitcher do not belong, and are likely the product of a transcription error in species_ID
# in the raw dataset during data entry... these rows will be dropped from the dataset.

working_df_subset = working_df_subset[working_df_subset.SPECIES_NAME != 'Long-billed Dowitcher']
working_df_subset = working_df_subset[working_df_subset.SPECIES_NAME != 'Common Tern']

counts = working_df_subset.groupby(['SPECIES_NAME', 'SPECIES_ID'])['BAND'].nunique()
print(counts)

SPECIES_NAME          SPECIES_ID
Black-bellied Plover  2700           2194
Piping Plover         2770          22597
Semipalmated Plover   2740          15454
Snowy Plover          2780           6867
Western Snowy Plover  2781             62
Wilson's Plover       2800           2794
Name: BAND, dtype: int64


In [43]:
# get unique values of day, month, year fields to ensure no abberant data or invalid data
days_unique = pd.unique(working_df['EVENT_DAY']) 
month_unique = pd.unique(working_df['EVENT_MONTH']) 
years_unique = pd.unique(working_df['EVENT_YEAR']) 
working_df_subset.drop(working_df_subset[working_df_subset['EVENT_MONTH'] > 12].index, inplace = True)
month_unique_fixed = pd.unique(working_df_subset['EVENT_MONTH']) 

print(days_unique)
print(month_unique) #obvious human error in the dataset - month '83' - this needs to be removed. will remove any values > 12.
print(years_unique)
print(month_unique_fixed)

[13 21  2 30 27 28 31 26 18 16 19 12 24 20  1 17 15 11 14  8  6  5 25 23
 29  9  3  7 22 10  4 99]
[ 6  8  5  7 12  9  4  3 10  1  2 11 83]
[2019 2016 2017 2018 2015 2014 2013 2012 2020 1998 1997 2009 2000 2001
 1999 2003 1994 1995 1996 1991 1992 2010 2008 1990 1989 1988 1987 2002
 2004 1993 2007 2006 2005 2011 1982 1986 1984 1983 1985 1981 1970 1971
 1977 1980 1978 1979 1974 1976 1975 1969 1972 1968 1966 1967 1965 1973
 1964 1963 1960 1962 1961]
[ 6  8  5  7 12  9  4  3 10  1  2 11]


In [44]:
# next step of process - counting and dealing with missing data
# get full number of rows for context

index = working_df_subset.index
number_of_rows = len(index) 
print('TOTAL ROWS:', number_of_rows)
print(working_df_subset.isna().sum())

TOTAL ROWS: 51227
BAND                             0
ORIGINAL_BAND                    0
OTHER_BANDS                  51055
BAND_TYPE                        0
BAND_STATUS                   1321
PRESENT_CONDITION_BAND       49906
BAND_CLOSURE_TYPE                0
EVENT_TYPE_DESCRIPTION           0
EVENT_DATE                       2
EVENT_DAY                        0
EVENT_MONTH                      0
EVENT_YEAR                       0
ISO_COUNTRY                      0
ISO_SUBDIVISION               2522
LAT_DD                           0
LON_DD                           0
COORDINATES_PRECISION            0
PERMIT                        1201
HOW_OBTAINED                 49906
WHO_OBTAINED                 49906
REPORTING_METHOD             49906
SPECIES_ID                       0
SPECIES_NAME                     0
BIRD_STATUS_DESCRIPTION       1203
AGE                           1204
SEX                           1205
MIN_AGE_AT_ENC                   0
SPECIES_ALPHA_CODE               0
TA

In [45]:
# for some columns like BAND_STATUS, HOW_OBTAINED, WHO_OBTAINED, REPORTING_METHOD, BIRD_STATUS_DESCRIPTION, AGE, 
# SEX, PRESENT_BIRD_CONDITION there are expected values, and the lack of a value indicates that the value is not known
# or was missed during data entry, hence we can replace missing values with "unknown"
working_df_subset[['BAND_STATUS', 'HOW_OBTAINED', 'WHO_OBTAINED', 'REPORTING_METHOD', 'BIRD_STATUS_DESCRIPTION',
                  'AGE', 'SEX', 'PRESENT_CONDITION_BIRD', 'PRESENT_CONDITION_BAND']] = working_df_subset[['BAND_STATUS', 'HOW_OBTAINED', 'WHO_OBTAINED',
                                                                                'REPORTING_METHOD', 'BIRD_STATUS_DESCRIPTION',
                                                                                'AGE', 'SEX', 'PRESENT_CONDITION_BIRD',
                                                                                'PRESENT_CONDITION_BAND']].fillna(value='Unknown')
print(working_df_subset.isna().sum())

BAND                             0
ORIGINAL_BAND                    0
OTHER_BANDS                  51055
BAND_TYPE                        0
BAND_STATUS                      0
PRESENT_CONDITION_BAND           0
BAND_CLOSURE_TYPE                0
EVENT_TYPE_DESCRIPTION           0
EVENT_DATE                       2
EVENT_DAY                        0
EVENT_MONTH                      0
EVENT_YEAR                       0
ISO_COUNTRY                      0
ISO_SUBDIVISION               2522
LAT_DD                           0
LON_DD                           0
COORDINATES_PRECISION            0
PERMIT                        1201
HOW_OBTAINED                     0
WHO_OBTAINED                     0
REPORTING_METHOD                 0
SPECIES_ID                       0
SPECIES_NAME                     0
BIRD_STATUS_DESCRIPTION          0
AGE                              0
SEX                              0
MIN_AGE_AT_ENC                   0
SPECIES_ALPHA_CODE               0
TAXONOMIC_ORDER     

In [46]:
# however for columns like PERMIT, OTHER_BANDS, ENDANGERED, AND ISO_SUBDIVISION, the missing data could also indicate a negative
# or false value (e.g. no permit, no other bands, or no ISO subdivision) -- we can check the unique values to determine if action is needed

other_bands_unique = pd.unique(working_df['OTHER_BANDS']) 
iso_subdivision_unique = pd.unique(working_df['ISO_SUBDIVISION']) 
permit_unique = pd.unique(working_df['PERMIT']) 
endangered_unique = pd.unique(working_df['ENDANGERED']) 

print(other_bands_unique) # Na values should be left as is - values are band identifiers 
print(iso_subdivision_unique) # NA values should be left as is - values are reserved for US states
# or canadian provinces and are not applicable to other countries
print(permit_unique) # NA values should be left as is - values are permit identifiers
print(endangered_unique) # Needs to be analyzed further.

[nan 'B67434680364;' 'B47434680096;' 'B28104218604;' 'B88074082320;'
 'B09034035610;' 'B59094589729;' 'B11914036409;' 'B31914035359;'
 'B91914779987;' 'B91914733993;' 'B99164382246;' 'B68764764426;'
 'B79064795245;' 'B48934744474;' 'B19164382254;' 'B48934744148;'
 'B02034793532;' 'B48934744134;' 'B48934744594;' 'B48934744513;'
 'B71914074854;' 'B41914072671;' 'B27694724536;' 'B01914080200;'
 'B11914080509;' 'B11914736023;' 'B01914701090;' 'B11914036481;'
 'B91914733917;' 'B01914710701;' 'B32034794743;' 'B77424980409;'
 'B29284212944;' 'B29034423573;' 'B09004019149;' 'B48934744358;'
 'B11914647627;' 'B71914765404;' 'B29034423414;' 'B72034793243;'
 'B01904267140;' 'B48164209312;' 'B99094589449;' 'B88934744404;'
 'B19284212938;' 'B19114214771;' 'B89114868944;' 'B48934744537;'
 'B99004019260; B38934744747;' 'B11914795701;' 'B01914035164;'
 'B88074082530;' 'B19094589115;' 'B99034423449;' 'B19114214699;'
 'B99034507684;' 'B78074082508;' 'B49074435887;' 'B89284212261;'
 'B48934744299;' 'B8865

In [47]:
# As shown above, the ENDANGERED field should be converted to boolean. Unique values are either nan or Y.
# We cannot trust that all nan values are equivalent to False, however. 
# To analyze, let us print all rows where species are listed as endangered. 
# Subject matter expertise would ideally be required to determine whether the endangered status referenced to local status

endangered_species_country_subdiv = working_df_subset.groupby(['ENDANGERED', 'SPECIES_NAME',
                                                              'ISO_COUNTRY', 'ISO_SUBDIVISION']).size().reset_index(name='Freq')
endangered_species_country_subdiv.head(100)

# the dataset shown below gives us all instances of where species are listed as endangered.
# this shows us that piping plover and western snowy plover are two species considered endangered in certain locations
# without subject matter expertise from the dataset owner, best to leave as is

,ENDANGERED,SPECIES_NAME,ISO_COUNTRY,ISO_SUBDIVISION,Freq
0,Y,Piping Plover,CA,CA-AB,936
1,Y,Piping Plover,CA,CA-MB,263
2,Y,Piping Plover,CA,CA-NB,494
3,Y,Piping Plover,CA,CA-NL,154
4,Y,Piping Plover,CA,CA-NS,626
5,Y,Piping Plover,CA,CA-ON,238
6,Y,Piping Plover,CA,CA-PE,295
7,Y,Piping Plover,CA,CA-QC,265
8,Y,Piping Plover,CA,CA-SK,3727
9,Y,Piping Plover,US,US-AL,94


In [48]:
# also note there are 2 dates missing in the EVENT_DATE field -- we can check to see if EVENT_DAY, EVENT_MONTH, and EVENT_YEAR
# are valid data points so we can reconstruct the EVENT_DATE

na_dates = working_df_subset[working_df_subset['EVENT_DATE'].isna()]
na_dates.head()

,BAND,ORIGINAL_BAND,OTHER_BANDS,BAND_TYPE,BAND_STATUS,PRESENT_CONDITION_BAND,BAND_CLOSURE_TYPE,EVENT_TYPE_DESCRIPTION,EVENT_DATE,EVENT_DAY,EVENT_MONTH,EVENT_YEAR,ISO_COUNTRY,ISO_SUBDIVISION,LAT_DD,LON_DD,COORDINATES_PRECISION,PERMIT,HOW_OBTAINED,WHO_OBTAINED,REPORTING_METHOD,SPECIES_ID,SPECIES_NAME,BIRD_STATUS_DESCRIPTION,AGE,SEX,MIN_AGE_AT_ENC,SPECIES_ALPHA_CODE,TAXONOMIC_ORDER,ENDANGERED,ALLOWABLE_BAND_SIZE,PRESENT_CONDITION_BIRD,EXTRA_INFO,RECORD_SOURCE_DESCRIPTION
5197,B09754900697,B09754900697,NaN,aluminum\butt end,Original band,Unknown,BE,Banding,NaN,99,5,1963,US,US-IA,40.91667,-91.58333,10 minute block,P9993979,Unknown,Unknown,Unknown,2740,Semipalmated Plover,Normal wild bird.,After Hatching Year,Unknown,nan,SEPL,304.0,NaN,"1A, 1B, 1D, 1, 1P",Unknown,NaN,Banding DB
41496,B87694717834,B87694717834,NaN,aluminum\butt-end (toll-free /web address),Unknown,LEFT ON BIRD,BE,Encounter,NaN,99,6,2012,US,US-CA,36.91667,-121.75000,10 minute block,NaN,Previously banded bird trapped and released du...,BIRD BANDERS,Reported on the Internet. This code was added ...,2781,Western Snowy Plover,Unknown,Unknown,Unknown,18,WSPL,301.0,Y,"1P, 1A, 1B",ALIVE - RELEASED,NaN,Encounter DB


In [49]:
# can fix this using the datetime module with format='%Y%j
na_dates['EVENT_DATE'] = pd.to_datetime(na_dates['EVENT_YEAR'] * 1000 + na_dates['EVENT_DAY'], format='%Y%j')
na_dates['EVENT_DATE'] = pd.to_datetime(na_dates['EVENT_DATE']).dt.strftime('%m/%d/%Y')
na_dates.head()

c:\users\mikea\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\mikea\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,BAND,ORIGINAL_BAND,OTHER_BANDS,BAND_TYPE,BAND_STATUS,PRESENT_CONDITION_BAND,BAND_CLOSURE_TYPE,EVENT_TYPE_DESCRIPTION,EVENT_DATE,EVENT_DAY,EVENT_MONTH,EVENT_YEAR,ISO_COUNTRY,ISO_SUBDIVISION,LAT_DD,LON_DD,COORDINATES_PRECISION,PERMIT,HOW_OBTAINED,WHO_OBTAINED,REPORTING_METHOD,SPECIES_ID,SPECIES_NAME,BIRD_STATUS_DESCRIPTION,AGE,SEX,MIN_AGE_AT_ENC,SPECIES_ALPHA_CODE,TAXONOMIC_ORDER,ENDANGERED,ALLOWABLE_BAND_SIZE,PRESENT_CONDITION_BIRD,EXTRA_INFO,RECORD_SOURCE_DESCRIPTION
5197,B09754900697,B09754900697,NaN,aluminum\butt end,Original band,Unknown,BE,Banding,04/09/1963,99,5,1963,US,US-IA,40.91667,-91.58333,10 minute block,P9993979,Unknown,Unknown,Unknown,2740,Semipalmated Plover,Normal wild bird.,After Hatching Year,Unknown,nan,SEPL,304.0,NaN,"1A, 1B, 1D, 1, 1P",Unknown,NaN,Banding DB
41496,B87694717834,B87694717834,NaN,aluminum\butt-end (toll-free /web address),Unknown,LEFT ON BIRD,BE,Encounter,04/08/2012,99,6,2012,US,US-CA,36.91667,-121.75000,10 minute block,NaN,Previously banded bird trapped and released du...,BIRD BANDERS,Reported on the Internet. This code was added ...,2781,Western Snowy Plover,Unknown,Unknown,Unknown,18,WSPL,301.0,Y,"1P, 1A, 1B",ALIVE - RELEASED,NaN,Encounter DB


In [50]:
# removing impacted rows from primary dataframe then re-adding from the subset copy using BAND as unique ID
# first we need to make sure there are no duplicate BAND values so no additional rows get deleted
working_df_subset.BAND.duplicated().sum 

<bound method Series.sum of 0        False
1        False
2        False
3        False
4        False
         ...  
51225    False
51226    False
51227    False
51228    False
51229    False
Name: BAND, Length: 51227, dtype: bool>

In [51]:
bands_to_correct_date = na_dates['BAND'].tolist()
working_df_subset = working_df_subset[~working_df_subset['BAND'].isin(bands_to_correct_date)]
working_df_subset.append(na_dates)

,BAND,ORIGINAL_BAND,OTHER_BANDS,BAND_TYPE,BAND_STATUS,PRESENT_CONDITION_BAND,BAND_CLOSURE_TYPE,EVENT_TYPE_DESCRIPTION,EVENT_DATE,EVENT_DAY,EVENT_MONTH,EVENT_YEAR,ISO_COUNTRY,ISO_SUBDIVISION,LAT_DD,LON_DD,COORDINATES_PRECISION,PERMIT,HOW_OBTAINED,WHO_OBTAINED,REPORTING_METHOD,SPECIES_ID,SPECIES_NAME,BIRD_STATUS_DESCRIPTION,AGE,SEX,MIN_AGE_AT_ENC,SPECIES_ALPHA_CODE,TAXONOMIC_ORDER,ENDANGERED,ALLOWABLE_BAND_SIZE,PRESENT_CONDITION_BIRD,EXTRA_INFO,RECORD_SOURCE_DESCRIPTION
0,B07104722502,B07104722502,NaN,Aluminum butt-end web address,Original band,Unknown,BE,Banding,6/13/2019,13,6,2019,US,US-FL,29.84389,-85.41472,exact,P6726807,Unknown,Unknown,Unknown,2800,Wilson's Plover,Normal wild bird.,Local,Unknown,nan,WIPL,302.0,NaN,"1D, 1A, 2",Unknown,NaN,Banding DB
1,B07104722582,B07104722582,NaN,Aluminum butt-end web address,Original band,Unknown,BE,Banding,6/21/2019,21,6,2019,US,US-FL,29.84389,-85.41472,exact,P6726807,Unknown,Unknown,Unknown,2800,Wilson's Plover,Normal wild bird.,Local,Unknown,nan,WIPL,302.0,NaN,"1D, 1A, 2",Unknown,NaN,Banding DB
2,B07234100054,B07234100054,NaN,aluminum\butt-end (toll-free /web address),Original band,Unknown,BE,Banding,8/2/2016,2,8,2016,US,US-ND,46.75000,-100.75000,10 minute block,P2962009,Unknown,Unknown,Unknown,2770,Piping Plover,Normal wild bird.,Local,Unknown,nan,PIPL,305.0,Y,"1A, 1B",Unknown,"Flag, streamer, or tab on leg.",Banding DB
3,B07234100061,B07234100061,NaN,aluminum\butt-end (toll-free /web address),Original band,Unknown,BE,Banding,6/2/2017,2,6,2017,US,US-ND,47.75000,-102.41667,10 minute block,P2962009,Unknown,Unknown,Unknown,2770,Piping Plover,Normal wild bird.,After Hatching Year,Unknown,nan,PIPL,305.0,Y,"1A, 1B",Unknown,Two or more types of auxiliary markers.,Banding DB
4,B07234100070,B07234100070,NaN,aluminum\butt-end (toll-free /web address),Original band,Unknown,BE,Banding,5/30/2017,30,5,2017,US,US-ND,47.91667,-102.41667,10 minute block,P2962009,Unknown,Unknown,Unknown,2770,Piping Plover,Normal wild bird.,After Hatching Year,Unknown,nan,PIPL,305.0,Y,"1A, 1B",Unknown,Two or more types of auxiliary markers.,Banding DB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51227,B99474003289,B99474003289,NaN,aluminum\butt end,Original band,Unknown,BE,Banding,12/31/1976,31,12,1976,JM,NaN,17.91667,-77.75000,10 minute block,P3737387,Unknown,Unknown,Unknown,2740,Semipalmated Plover,Normal wild bird.,Unknown,Unknown,nan,SEPL,304.0,NaN,"1A, 1B, 1D, 1, 1P",Unknown,NaN,Banding DB
51228,B99474143763,B99474143763,NaN,aluminum\butt end,Original band,Unknown,BE,Banding,6/9/1960,9,6,1960,US,US-AK,63.08333,-145.58333,10 minute block,P9954904,Unknown,Unknown,Unknown,2740,Semipalmated Plover,Normal wild bird.,Unknown,Unknown,nan,SEPL,304.0,NaN,"1A, 1B, 1D, 1, 1P",Unknown,NaN,Banding DB
51229,B99474959349,B99474959349,NaN,aluminum\butt end,Original band,Unknown,BE,Banding,9/20/1965,20,9,1965,US,US-MA,42.08333,-70.58333,10 minute block,P9961209,Unknown,Unknown,Unknown,2740,Semipalmated Plover,Normal wild bird.,Unknown,Unknown,nan,SEPL,304.0,NaN,"1A, 1B, 1D, 1, 1P",Unknown,Blood sample taken.,Banding DB
5197,B09754900697,B09754900697,NaN,aluminum\butt end,Original band,Unknown,BE,Banding,04/09/1963,99,5,1963,US,US-IA,40.91667,-91.58333,10 minute block,P9993979,Unknown,Unknown,Unknown,2740,Semipalmated Plover,Normal wild bird.,After Hatching Year,Unknown,nan,SEPL,304.0,NaN,"1A, 1B, 1D, 1, 1P",Unknown,NaN,Banding DB


In [52]:
# convert the entire column to datetime
working_df_subset['EVENT_DATE'] = pd.to_datetime(working_df_subset['EVENT_DATE'], format= '%m/%d/%Y')

In [53]:
# one more check of datetypes to ensure types are correct
working_df_subset.dtypes

BAND                                 object
ORIGINAL_BAND                        object
OTHER_BANDS                          object
BAND_TYPE                            object
BAND_STATUS                          object
PRESENT_CONDITION_BAND               object
BAND_CLOSURE_TYPE                    object
EVENT_TYPE_DESCRIPTION               object
EVENT_DATE                   datetime64[ns]
EVENT_DAY                             int64
EVENT_MONTH                           int64
EVENT_YEAR                            int64
ISO_COUNTRY                          object
ISO_SUBDIVISION                      object
LAT_DD                              float64
LON_DD                              float64
COORDINATES_PRECISION                object
PERMIT                               object
HOW_OBTAINED                         object
WHO_OBTAINED                         object
REPORTING_METHOD                     object
SPECIES_ID                           object
SPECIES_NAME                    

In [54]:
# note the taxonomic order is a float - should be an integer as it is represented as a whole number
working_df_subset['TAXONOMIC_ORDER'] = working_df_subset['TAXONOMIC_ORDER'].astype(int)
working_df_subset.head()

,BAND,ORIGINAL_BAND,OTHER_BANDS,BAND_TYPE,BAND_STATUS,PRESENT_CONDITION_BAND,BAND_CLOSURE_TYPE,EVENT_TYPE_DESCRIPTION,EVENT_DATE,EVENT_DAY,EVENT_MONTH,EVENT_YEAR,ISO_COUNTRY,ISO_SUBDIVISION,LAT_DD,LON_DD,COORDINATES_PRECISION,PERMIT,HOW_OBTAINED,WHO_OBTAINED,REPORTING_METHOD,SPECIES_ID,SPECIES_NAME,BIRD_STATUS_DESCRIPTION,AGE,SEX,MIN_AGE_AT_ENC,SPECIES_ALPHA_CODE,TAXONOMIC_ORDER,ENDANGERED,ALLOWABLE_BAND_SIZE,PRESENT_CONDITION_BIRD,EXTRA_INFO,RECORD_SOURCE_DESCRIPTION
0,B07104722502,B07104722502,NaN,Aluminum butt-end web address,Original band,Unknown,BE,Banding,2019-06-13,13,6,2019,US,US-FL,29.84389,-85.41472,exact,P6726807,Unknown,Unknown,Unknown,2800,Wilson's Plover,Normal wild bird.,Local,Unknown,nan,WIPL,302,NaN,"1D, 1A, 2",Unknown,NaN,Banding DB
1,B07104722582,B07104722582,NaN,Aluminum butt-end web address,Original band,Unknown,BE,Banding,2019-06-21,21,6,2019,US,US-FL,29.84389,-85.41472,exact,P6726807,Unknown,Unknown,Unknown,2800,Wilson's Plover,Normal wild bird.,Local,Unknown,nan,WIPL,302,NaN,"1D, 1A, 2",Unknown,NaN,Banding DB
2,B07234100054,B07234100054,NaN,aluminum\butt-end (toll-free /web address),Original band,Unknown,BE,Banding,2016-08-02,2,8,2016,US,US-ND,46.75000,-100.75000,10 minute block,P2962009,Unknown,Unknown,Unknown,2770,Piping Plover,Normal wild bird.,Local,Unknown,nan,PIPL,305,Y,"1A, 1B",Unknown,"Flag, streamer, or tab on leg.",Banding DB
3,B07234100061,B07234100061,NaN,aluminum\butt-end (toll-free /web address),Original band,Unknown,BE,Banding,2017-06-02,2,6,2017,US,US-ND,47.75000,-102.41667,10 minute block,P2962009,Unknown,Unknown,Unknown,2770,Piping Plover,Normal wild bird.,After Hatching Year,Unknown,nan,PIPL,305,Y,"1A, 1B",Unknown,Two or more types of auxiliary markers.,Banding DB
4,B07234100070,B07234100070,NaN,aluminum\butt-end (toll-free /web address),Original band,Unknown,BE,Banding,2017-05-30,30,5,2017,US,US-ND,47.91667,-102.41667,10 minute block,P2962009,Unknown,Unknown,Unknown,2770,Piping Plover,Normal wild bird.,After Hatching Year,Unknown,nan,PIPL,305,Y,"1A, 1B",Unknown,Two or more types of auxiliary markers.,Banding DB


In [55]:
# case standardization - lower
working_df_subset['BAND_TYPE'] = working_df_subset['BAND_TYPE'].str.lower()
working_df_subset['BAND_STATUS'] = working_df_subset['BAND_STATUS'].str.lower()
working_df_subset['PRESENT_CONDITION_BAND'] = working_df_subset['PRESENT_CONDITION_BAND'].str.lower()
working_df_subset['EVENT_TYPE_DESCRIPTION'] = working_df_subset['EVENT_TYPE_DESCRIPTION'].str.lower()
working_df_subset['COORDINATES_PRECISION'] = working_df_subset['COORDINATES_PRECISION'].str.lower()
working_df_subset['HOW_OBTAINED'] = working_df_subset['HOW_OBTAINED'].str.lower()
working_df_subset['WHO_OBTAINED'] = working_df_subset['WHO_OBTAINED'].str.lower()
working_df_subset['REPORTING_METHOD'] = working_df_subset['REPORTING_METHOD'].str.lower()
working_df_subset['SPECIES_NAME'] = working_df_subset['SPECIES_NAME'].str.lower()
working_df_subset['BIRD_STATUS_DESCRIPTION'] = working_df_subset['BIRD_STATUS_DESCRIPTION'].str.lower()
working_df_subset['AGE'] = working_df_subset['AGE'].str.lower()
working_df_subset['SEX'] = working_df_subset['SEX'].str.lower()
working_df_subset['ENDANGERED'] = working_df_subset['ENDANGERED'].str.lower()
working_df_subset['PRESENT_CONDITION_BIRD'] = working_df_subset['PRESENT_CONDITION_BIRD'].str.lower()
working_df_subset['EXTRA_INFO'] = working_df_subset['EXTRA_INFO'].str.lower()


# case standardization - upper
working_df_subset['BAND'] = working_df_subset['BAND'].str.upper()
working_df_subset['ORIGINAL_BAND'] = working_df_subset['ORIGINAL_BAND'].str.upper()
working_df_subset['OTHER_BANDS'] = working_df_subset['OTHER_BANDS'].str.upper()
working_df_subset['BAND_CLOSURE_TYPE'] = working_df_subset['BAND_CLOSURE_TYPE'].str.upper()
working_df_subset['ISO_COUNTRY'] = working_df_subset['ISO_COUNTRY'].str.upper()
working_df_subset['ISO_SUBDIVISION'] = working_df_subset['ISO_SUBDIVISION'].str.upper()
working_df_subset['ALLOWABLE_BAND_SIZE'] = working_df_subset['ALLOWABLE_BAND_SIZE'].str.upper()

In [56]:
working_df_subset.to_csv('Output_Files/cleaned_banding_data_output.csv', index=False)